## Exercício sobre pytorch

Utilize o dataset 'datasetCarros.csv'.<br>
Usando Pytorch, construa uma rede neural para prever a feature 'PrecoVenda'.<br>

Use uma rede neural feed forward com duas camadas escondidas, com 20 neurônios cada.<br>
Use o critério de perda MSELoss, otimizador Adam e learning rate = 0.001. Considere 1000 épocas.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
import torch
import numpy as np

class Feedforward(torch.nn.Module):
    
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, 1)
            
            self.relu = torch.nn.ReLU()
            
            
        def forward(self, x):
            output = self.fc1(x)
            output = self.relu(output)
            
            output = self.fc2(output)
            output = self.relu(output)
            
            output = self.fc3(output)

            return output

device = "cuda" if torch.cuda.is_available() else "cpu"

dfCarros = pd.read_csv('datasetCarros.csv')

dfCarros.replace({'TipoCombustivel':{'Petrol':0,'Diesel':1,'CNG':2}},inplace=True)
dfCarros.replace({'Trasmissao':{'Manual':0,'Automatic':1}},inplace=True)


y= dfCarros['PrecoVenda']
X = dfCarros.drop(['Nome','PrecoVenda'],axis=1)


y_tensor = torch.tensor(y)
X_tensor = torch.tensor(X.to_numpy())
print(y_tensor.shape)
print(X_tensor.shape)

X_treino, X_teste, y_treino, y_teste = train_test_split(X_tensor, y_tensor, test_size = 0.10, random_state=5)

print(X_treino.shape)
print(y_treino.shape)

X_treino = X_treino.float().to(device)
y_treino = y_treino.float().to(device)

X_teste = X_teste.float().to(device)
y_teste = y_teste.float().to(device)


model = Feedforward(6, 20).to(device)
print(model)
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


model.eval()
y_pred = model(X_teste)
antes_treino = criterion(y_pred, y_teste) 
print('Teste - perda antes do treinamento' , antes_treino.item())

model.train()
epoch = 100

for epoch in range(epoch):
    optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(X_treino)
    
    # Computa a perda
    loss = criterion(y_pred, y_treino)
    
    print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Passe de Backward
    loss.backward()
    optimizer.step()

    
model.eval()
y_pred = model(X_teste)
after_train = criterion(y_pred, y_teste) 
print('Teste - perda depois do treinamento' , after_train.item())    

torch.Size([301])
torch.Size([301, 6])
torch.Size([270, 6])
torch.Size([270])
Feedforward(
  (fc1): Linear(in_features=6, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
  (relu): ReLU()
)
Teste - perda antes do treinamento 3993110.75
Epoch 0: perda treino: 3016028.5
Epoch 1: perda treino: 2248270.5
Epoch 2: perda treino: 1595616.125
Epoch 3: perda treino: 1060782.25
Epoch 4: perda treino: 643440.5
Epoch 5: perda treino: 351430.0
Epoch 6: perda treino: 159260.359375
Epoch 7: perda treino: 48693.55859375
Epoch 8: perda treino: 5468.8017578125
Epoch 9: perda treino: 12129.5849609375
Epoch 10: perda treino: 50966.47265625
Epoch 11: perda treino: 105692.5625
Epoch 12: perda treino: 161547.53125
Epoch 13: perda treino: 207440.390625
Epoch 14: perda treino: 236744.21875
Epoch 15: perda treino: 247080.609375
Epoch 16: perda treino: 239431.203125
Epoch 17: perda treino: 217071.625
Epoch 18: perda

/home/dnl/.local/share/virtualenvs/exercicio_5-_7hTz5eM/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/dnl/.local/share/virtualenvs/exercicio_5-_7hTz5eM/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([270])) that is different to the input size (torch.Size([270, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
